<h1> DIPS: a Detection Itinerary Planner for transiting Systems </h1>

NOTE: The date picker widget does not work in Safari. To open Jupyter notebook in another browser, see instructions [here](https://medium.com/@tyagi.sudarshini/change-default-browser-in-jupyter-notebook-mac-667b56a3274e).

In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

Click button below to start or restart (refresh) notebook:

In [15]:
from IPython.display import HTML

#::: globals
global INPUT
global VBOXES
global BUTTONS
global DROPDOWNS
INPUT = {}
VBOXES = {}
BUTTONS = {}
DROPDOWNS = {}
layout = {'width': '180px'}
layout_wide = {'width': '360px'}
layout_textbox = {'width': '120px'}
layout_checkbox = {}

import sys
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages')
import numpy as np
from astropy.table import Table
import datetime
import ipydatetime
from astropy import time, coordinates as coord, units as u
from prettytable import PrettyTable
from datetime import date, timedelta
from operator import itemgetter
import csv
import shutil
import os
import ipywidgets as widgets
from ipywidgets import interact, interactive_output, interactive
from IPython.display import display, Markdown, clear_output, Javascript
display(HTML("<style>.container { width:80% !important; }</style>"))
from tkinter import Tk, filedialog

layout = widgets.Layout(width='auto', height='40px') #set width and height

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

button = widgets.Button(
    description="Click to set up notebook",
    display='flex',
    flex_flow='column',
    align_items='stretch', 
    layout = layout)
button.on_click(run_all)
display(button)

<IPython.core.display.Javascript object>

Button(description='Click to set up notebook', layout=Layout(height='40px', width='auto'), style=ButtonStyle()…

------------

<h3>You will need: </h3>

<br/>

- <h4>A txt file of your ephemerides with columns in the following order: </h4>
<br/>

    - Target name
    - Epoch (JD)
    - Period (days)
    - Transit width (days)
    - RA (hr)
    - RA (min)
    - RA (sec)
    - Dec (deg)
    - Dec (min)
    - Dec (sec)


- <h4>The RA range over which you want to observe </h4>
- <h4>Sunset/ sunrise (or astronomical twilight start/ end) times </h4>
- <h4>Start/ end dates of run </h4>
<br/>

    - NOTE: End date should be the date of the final morning when observations will be taken (i.e. NOT night beginning date)

- <h4>Longitude of observatory </h4>
<br/>

    - NOTE: Positive values are in the Eastern hemisphere, negative values are in the Western hemisphere
    - e.g. La Silla: -70.738 (W)
    - e.g. SAAO 1m: 20.8101(E)
-------

<h4>Do you want to find transits for planets, binaries or both?</h4>

In [16]:
checkbox_list = []
inc_secondary_transits = []

def yes_checkbox(Yes):
    if Yes == True:
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
        inc_secondary_transits.append('yes')
        try:
            inc_secondary_transits.remove('no')
        except:
            pass
    if Yes == False:
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
        inc_secondary_transits.append('no')
        try:
            inc_secondary_transits.remove('yes')
        except:
            pass

yes_no = interactive(yes_checkbox, Yes= False)

def planets_checkbox(Planets):
    if Planets == True:
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
        checkbox_list.append('planets')
    if Planets == False:
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
        try:
            checkbox_list.remove('planets')
        except:
            pass
        
def binaries_checkbox(Binaries):
    binaries_checkbox = widgets.Checkbox(description = 'Binaries', indent=False)
    if Binaries == True:
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
        checkbox_list.append('binaries')
        checkbox2_list = []
        print(' ')
        display(Markdown('Do you want to find primary AND secondary binary transits?'))
        display(yes_no)
        #yes_checkbox = widgets.Checkbox(description = 'Yes')
        no_checkbox = widgets.Checkbox(description = 'No')
        #checkbox2_list.append(yes_checkbox)
        checkbox2_list.append(no_checkbox)
        display(widgets.VBox(checkbox2_list))
    if Binaries == False:
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
        try:
            checkbox_list.remove('binaries')
        except:
            pass
        
#planets_checkbox = widgets.Checkbox(description = 'Planets')
#binaries_checkbox = widgets.Checkbox(description = 'Binaries')
#thing = interact(binaries_checkbox, value=False)
#display(planets_checkbox)

planets = interactive(planets_checkbox, Planets=False)
binaries = interactive(binaries_checkbox, Binaries=False)


display(planets)
display(binaries)

interactive(children=(Checkbox(value=False, description='Planets'), Output()), _dom_classes=('widget-interact'…

interactive(children=(Checkbox(value=False, description='Binaries'), Output()), _dom_classes=('widget-interact…

<h4>Choose the data file(s):</h4>

In [17]:
hbox_list = []

def select_data(change):
        root = Tk()
        root.withdraw()
        root.call('wm', 'attributes', '.', '-topmost', True)
        INPUT['data'] = filedialog.askopenfilename(multiple=False)
        %gui tk
        if INPUT['data'] != '':
            text_af_directory.value = INPUT['data']
            BUTTONS['data'].style.button_color = 'lightgreen'
            INPUT['show_step_2a'] = True
            display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))

def select_data2(change):
        root = Tk()
        root.withdraw()
        root.call('wm', 'attributes', '.', '-topmost', True)
        INPUT['data2'] = filedialog.askopenfilename(multiple=False)
        %gui tk
        if INPUT['data2'] != '':
            text_af_directory2.value = INPUT['data2']
            BUTTONS['data2'].style.button_color = 'lightgreen'
            INPUT['show_step_2b'] = True
            display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))

            
if ('planets' in checkbox_list) and ('binaries' in checkbox_list):
    
    BUTTONS['data'] = widgets.Button(description='Select planets file', button_style='')
    text_af_directory = widgets.Text(value='', placeholder='for example: Planet_ephemerides.txt', disable=True)
    hbox = widgets.HBox([BUTTONS['data'], text_af_directory])
    hbox_list.append(hbox)
    
    BUTTONS['data2'] = widgets.Button(description='Select binaries file', button_style='')
    text_af_directory2 = widgets.Text(value='', placeholder='for example: Binary_ephemerides.txt', disable=True)
    hbox2 = widgets.HBox([BUTTONS['data2'], text_af_directory2])
    hbox_list.append(hbox2)
    
    BUTTONS['data'].on_click(select_data)
    BUTTONS['data2'].on_click(select_data2)
    
    display(widgets.VBox(hbox_list))
    
else:
    
    BUTTONS['data'] = widgets.Button(description='Select file', button_style='')
    text_af_directory = widgets.Text(value='', placeholder='for example: Ephemerides.txt', disable=True)
    hbox = widgets.HBox([BUTTONS['data'], text_af_directory])
    display(hbox)

    BUTTONS['data'].on_click(select_data)

In [18]:
#text = widgets.Text(
#    placeholder='Type something',
#    description='String:',
#    disabled=False
#)

#display(text)
if ('planets' in checkbox_list) and ('binaries' in checkbox_list):
    if 'show_step_2a' in INPUT and INPUT['show_step_2a'] == True:
        load_file_planets = (np.loadtxt("%s"%INPUT['data'], delimiter=','))
    if 'show_step_2b' in INPUT and INPUT['show_step_2b'] == True:
        load_file_binaries = (np.loadtxt("%s"%INPUT['data2'], delimiter=','))
else:
    if 'show_step_2a' in INPUT and INPUT['show_step_2a'] == True:
        load_file = (np.loadtxt("%s"%INPUT['data'], delimiter=','))

#text.on_submit(callback)

In [19]:
def NGTS_orion_to_HJD(NGTS_orion_date):
    hjd_0 = 2450000.000000 # 1995-10-09 12:00:00.00 converted to JD                     
    return NGTS_orion_date + hjd_0

def obs_window(start, end):
    #salt = coord.EarthLocation.of_site('salt')                                         
    obs_times = [start, end]
    obs_window = time.Time(obs_times, format='isot', scale='utc')                                                                   
    obs_window = obs_window.jd
    return obs_window

def observable_transits(period, epoch, width, obs_window, utc_offset):
    transit = epoch
    times = [transit]
    num = [1]
    kind = []
    while transit < obs_window[1]:
        num.append(num[-1] +1)
        transit += period
        times.append(transit)
    num = np.asarray(num)
    num = num[(times > obs_window[0]) & (times < obs_window[1])]
    for i in num:
        if i % 2 == 0:
            kind.append("even")
        else:
            kind.append("odd")
    times = [i for i in times if i > obs_window[0] and i < obs_window[1]]
    times = np.asarray(times) + utc_offset/24.0
    ingress = times - 0.5*width/24.0
    egress = times + 0.5*width/24.0
    times = time.Time(times, format='jd', scale='utc', out_subfmt='date_hm').iso
    ingress = time.Time(ingress, format='jd', scale='utc',
            out_subfmt='date_hm').iso
    egress = time.Time(egress, format='jd', scale='utc',
            out_subfmt='date_hm').iso
    return(ingress, times, egress, kind)  

def split_times(ingress, egress, i):
    time_in = ingress[i].split()[1]
    time_in_hr = int(time_in.split(':')[0])
    time_in_min = int(time_in.split(':')[1])
    time_eg = egress[i].split()[1]
    time_eg_hr = int(time_eg.split(':')[0])
    time_eg_min = int(time_eg.split(':')[1])
    ingress_time = datetime.time(hour=time_in_hr, minute=time_in_min)
    egress_time = datetime.time(hour=time_eg_hr, minute=time_eg_min)
    return(ingress_time, egress_time)

# Only select transits occuring at night:
def night_times_print(ingress, egress, times, kind, noi_number, period, epoch, width, RA_hr, RA_m, 
                      RA_sec, utc_offset, sunset, sunrise, transit_times_full_list, last_row, Dec_deg, Dec_min, Dec_sec):
    
    RA = "%s:%s:%s" %(format(RA_hr, '02.0f'), format(RA_m, '.0f'), format(RA_sec, '.0f'))
    Dec = "%s:%s:%s" %(format(Dec_deg, '02.0f'), format(Dec_min, '02.0f'), format(Dec_sec, '02.0f'))
    
    if last_row == True:
        return(transit_times_full_list)
    
    if last_row == False:
        new_ingress_list = []
        new_times_list = []
        new_egress_list = []
        new_kind_list = []
        for i in range(len(ingress)):
            # Get times in useable format:
            ingress_time = split_times(ingress, egress, i)[0]
            egress_time = split_times(ingress, egress, i)[1]
            # Choose transits where ingress/ egress are after sunset or before sunrise:
            if ((ingress_time >= sunset) or (ingress_time <= sunrise)
                or (egress_time >= sunset) or (egress_time <= sunrise)):
                new_ingress_list.append(ingress[i])
                new_times_list.append(times[i])
                new_egress_list.append(egress[i])
                new_kind_list.append(kind[i])
                # Spit out a list of every transit and the parameters
                transit_times_full_list.append((noi_number, RA, Dec, ingress[i], times[i], egress[i], kind[i]))
    
        # Make table of these results and print:
        t = PrettyTable()
        t.add_column("ingress (LT)", new_ingress_list)
        t.add_column("mid_transit (LT)", new_times_list)
        t.add_column("egress (LT)", new_egress_list)
        t.add_column("Odd/even", new_kind_list)

        print("\n")
        print(noi_number)
        print("\nperiod (days): %s" %period)
        print("epoch (HJD): %s" %epoch)
        print("width (hours): %s" %width)
        print("RA: %s"%RA)
        print("Dec: %s" %Dec)
        print("\nPRIMARY transit times UTC+%s hr:" %utc_offset)
        print(t)
        print("\n")

def get_all_transits(file, start_date, end_date, utc_offset, RA_min, RA_max,
                     sunset_time, sunrise_time, primary_secondary):
    
    obs_start   = "%sT12:00:00" %(start_date)# UTC                                               
    obs_end     = "%sT12:00:00" %(end_date) # UTC                                               
    #utc_offset = +2 # offset from UTC to LT in hours
    
    transit_times_full_list = []

    for index, row in enumerate(file):
        noi_number = "NOI-" + str("%.0f" %row[0])
        period = row[2]
        epoch = row[1] - 2450000
        width = row[3]
        RA_hr = row[4]
        RA_m = row[5]
        RA_sec = row[6]
        Dec_deg = row[7]
        Dec_min = row[8]
        Dec_sec = row[9]
        
        # Conversions:                                                                       
        epoch = NGTS_orion_to_HJD(epoch)

        # Get observing window in JD:                                                            
        window = obs_window(obs_start, obs_end)
        
        # Put sunrise/ sunset into useable format:
        sunset = datetime.time(hour=int(sunset_time.split(':')[0]), minute=int(sunset_time.split(':')[1]))
        sunrise = datetime.time(hour=int(sunrise_time.split(':')[0]), minute=int(sunrise_time.split(':')[1]))
        
        if primary_secondary == 'primary':
            try:
                ingress, times, egress, kind = observable_transits(period, epoch, width, window, utc_offset)
            except:
                ingress = 0
                times = ''
                
        elif primary_secondary == 'secondary':
            try:
                ingress, times, egress, kind = observable_transits(period, epoch+0.5*period, width, window, utc_offset)
            except:
                ingress = 0
                times = ''
            
        # If observable RAs run from e.g. 22hr - 5hr:
        if RA_min > RA_max:
            if ((RA_hr >= RA_min) or (RA_hr < RA_max)) and (ingress != 0):
                night_times_print(ingress, egress, times, kind, noi_number, period, epoch, width, RA_hr,
                                RA_m, RA_sec, utc_offset, sunset, sunrise, transit_times_full_list, False, Dec_deg, Dec_min, Dec_sec)
                
        elif RA_min < RA_max:
            if ((RA_hr >= RA_min) and (RA_hr < RA_max)) and (ingress != 0):
                night_times_print(ingress, egress, times, kind, noi_number, period, epoch, width, RA_hr,
                                RA_m, RA_sec, utc_offset, sunset, sunrise, transit_times_full_list, False, Dec_deg, Dec_min, Dec_sec)
                
        if index == (len(file)-1):
            all_transits = night_times_print(ingress, egress, times, kind, noi_number, period, epoch, width, RA_hr,
                                RA_m, RA_sec, utc_offset, sunset, sunrise, transit_times_full_list, True, Dec_deg, Dec_min, Dec_sec)
            return(all_transits)

<h4>Find the transits which occur at night for each object over your date range:</h4>

In [20]:
Date_list = []
widget_list = []
RA_list = []
time_list = []

start_date_val = widgets.DatePicker(description='Start date:',disabled=False)
Date_list.append(start_date_val)
end_date_val = widgets.DatePicker(description='End date:',disabled=False)
Date_list.append(end_date_val)

text_utc_offset = widgets.BoundedIntText(min=-12,max=14,description='UTC offset:',disabled=False)
widget_list.append(widgets.HBox([text_utc_offset]))

longitude_widget = widgets.FloatText(description='Longitude:',disabled=False)
widget_list.append(widgets.HBox([longitude_widget]))

RA_selection_lower = widgets.BoundedIntText(min=0,max=23,description='Start RA:',disabled=False)
RA_list.append(RA_selection_lower)
RA_selection_upper = widgets.BoundedIntText(min=0,max=23,description='End RA:',disabled=False)
RA_list.append(RA_selection_upper)

sunset_time = ipydatetime.TimePicker(description='Sunset time:')
time_list.append(sunset_time)
sunrise_time = ipydatetime.TimePicker(description='Sunrise time:')
time_list.append(sunrise_time)

display(widgets.HBox(Date_list))
display(widgets.VBox(widget_list))
display(widgets.HBox(RA_list))
display(widgets.HBox(time_list))

In [21]:
def run_cell(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+4)'))

button_find_transits = widgets.Button(description="Find transits")
button_find_transits.on_click(run_cell)
display(button_find_transits)

Button(description='Find transits', style=ButtonStyle())

In [22]:
all_transits_list = []

try:
    sunrise_time
except NameError:
    sunrise_time = None
    
if 'None' in str(sunrise_time):
    pass

elif ('planets' in checkbox_list) and ('binaries' in checkbox_list) and ('yes' in inc_secondary_transits):
    
    sunset = str(sunset_time.value)
    sunrise = str(sunrise_time.value)
    all_transits_planets = get_all_transits(load_file_planets, '%s'%(start_date_val.value), '%s'%(end_date_val.value),
                                    text_utc_offset.value, RA_selection_lower.value, RA_selection_upper.value,
                                    '%s'%(sunset[:-3]), '%s'%(sunrise[:-3]), 'primary')
    all_transits_binaries_primary = get_all_transits(load_file_binaries, '%s'%(start_date_val.value), '%s'%(end_date_val.value),
                                    text_utc_offset.value, RA_selection_lower.value, RA_selection_upper.value,
                                    '%s'%(sunset[:-3]), '%s'%(sunrise[:-3]), 'primary')
    all_transits_binaries_secondary = get_all_transits(load_file_binaries, '%s'%(start_date_val.value), '%s'%(end_date_val.value),
                                    text_utc_offset.value, RA_selection_lower.value, RA_selection_upper.value,
                                    '%s'%(sunset[:-3]), '%s'%(sunrise[:-3]), 'secondary')
        
    all_transits_list.append('planets')
    all_transits_list.append(all_transits_planets)
    all_transits_list.append('binaries_primary')
    all_transits_list.append(all_transits_binaries_primary)
    all_transits_list.append('binaries_secondary')
    all_transits_list.append(all_transits_binaries_secondary)
    
elif ('planets' in checkbox_list) and ('binaries' in checkbox_list) and ('yes' not in inc_secondary_transits):

    sunset = str(sunset_time.value)
    sunrise = str(sunrise_time.value)
    all_transits_planets = get_all_transits(load_file_planets, '%s'%(start_date_val.value), '%s'%(end_date_val.value),
                                    text_utc_offset.value, RA_selection_lower.value, RA_selection_upper.value,
                                    '%s'%(sunset[:-3]), '%s'%(sunrise[:-3]), 'primary')
    all_transits_binaries_primary = get_all_transits(load_file_binaries, '%s'%(start_date_val.value), '%s'%(end_date_val.value),
                                    text_utc_offset.value, RA_selection_lower.value, RA_selection_upper.value,
                                    '%s'%(sunset[:-3]), '%s'%(sunrise[:-3]), 'primary')
        
    all_transits_list.append('planets')
    all_transits_list.append(all_transits_planets)
    all_transits_list.append('binaries_primary')
    all_transits_list.append(all_transits_binaries_primary)
    
elif (('planets' in checkbox_list) or ('binaries' in checkbox_list)) and ('yes' in inc_secondary_transits):
    sunset = str(sunset_time.value)
    sunrise = str(sunrise_time.value)
    all_transits_primary = get_all_transits(load_file, '%s'%(start_date_val.value), '%s'%(end_date_val.value),
                                    text_utc_offset.value, RA_selection_lower.value, RA_selection_upper.value,
                                    '%s'%(sunset[:-3]), '%s'%(sunrise[:-3]), 'primary')
    all_transits_secondary = get_all_transits(load_file, '%s'%(start_date_val.value), '%s'%(end_date_val.value),
                                    text_utc_offset.value, RA_selection_lower.value, RA_selection_upper.value,
                                    '%s'%(sunset[:-3]), '%s'%(sunrise[:-3]), 'secondary')
    
    all_transits_list.append('%s_primary'%(checkbox_list[0]))
    all_transits_list.append(all_transits_primary)
    all_transits_list.append('%s_secondary'%(checkbox_list[0]))
    all_transits_list.append(all_transits_secondary)

else:
    sunset = str(sunset_time.value)
    sunrise = str(sunrise_time.value)
    all_transits = get_all_transits(load_file, '%s'%(start_date_val.value), '%s'%(end_date_val.value),
                                    text_utc_offset.value, RA_selection_lower.value, RA_selection_upper.value,
                                    '%s'%(sunset[:-3]), '%s'%(sunrise[:-3]), 'primary')
    all_transits_list.append(all_transits)

In [23]:
# Hour angle calculator

def LST_finder(year, month, day, time_UTC, longitude, in_or_eg):
    JD = (367*year) - int((7*(year+int((month+9)/12)))/4) + int((275*month)/9) + day + 1721013.5 + (time_UTC/24)
    #print('\n%s Julian Date: JD%s' %(in_or_eg, JD))

    #calculate the Greenwhich mean sidereal time:
    GMST = 18.697374558 + 24.06570982441908*(JD - 2451545)
    GMST = GMST % 24    #use modulo operator to convert to 24 hours
    GMSTmm = (GMST - int(GMST))*60          #convert fraction hours to minutes
    GMSTss = (GMSTmm - int(GMSTmm))*60      #convert fractional minutes to seconds
    GMSThh = int(GMST)
    GMSTmm = int(GMSTmm)
    GMSTss = int(GMSTss)
    #print('\nGreenwhich Mean Sidereal Time: %s:%s:%s' %(GMSThh, GMSTmm, GMSTss))

    #Convert to the local sidereal time by adding the longitude (in hours) from the GMST.
    #(Hours = Degrees/15, Degrees = Hours*15)
    longitude = longitude/15      #Convert longitude to hours
    LST = GMST+longitude     #Fraction LST. If negative we want to add 24...
    if LST < 0:
        LST = LST +24
    #LSTmm = (LST - int(LST))*60          #convert fraction hours to minutes
    #LSTss = (LSTmm - int(LSTmm))*60      #convert fractional minutes to seconds
    #LSThh = int(LST)
    #LSTmm = int(LSTmm)
    #LSTss = int(LSTss)

    return LST

def HA_finder(date_in, RA_obj, ingress, egress, utc_offset, longitude):
    
    #Sidereal Time and Julian Date Calculator
    #Revision history: Justine Haupt, v1.0 (11/23/17)

    #Only valid for dates between 1901 and 2099. Accurate to within 1.1s.
    
    #References:
    #http://aa.usno.navy.mil/faq/docs/JD_Formula.php
    #http://aa.usno.navy.mil/faq/docs/GAST.php and
    #Longitude of location in question (BMX LAT = 40.869 [40deg 52' 8"], BMX LONG = -72.866 [-72deg 51' 57"], Custer LONG = -72.435)

    #Calculate longitude in DegHHMM format for edification of user:
    hemisphere = 'W'
    if longitude > 0:        #if the number is positive it's in the Eastern hemisphere
        hemisphere = 'E'
    LongDeg = int(longitude)
    LongMin = (longitude - int(longitude))*60
    LongSec = (LongMin - int(LongMin))*60
    LongMin = int(LongMin)
    LongSec = int(LongSec)

    in_hr, in_min = ingress.split(':')
    eg_hr, eg_min = egress.split(':')
    
    in_hr = int(in_hr)
    in_min = int(in_min)
    eg_hr = int(eg_hr)
    eg_min = int(eg_min)
        
    if utc_offset != 0:
        in_hr = in_hr - utc_offset
        eg_hr = eg_hr - utc_offset
    
    year_in,month_in,day_in = date_in.split('-')
    
    # Make egress date the next day if egress is in morning and ingress is in evening
    if in_hr > eg_hr:
        date_in = datetime.date(int(year_in),int(month_in),int(day_in))
        date_eg = date_in + datetime.timedelta(days=1)
        date_eg_str = str(date_eg)
        year_eg,month_eg,day_eg = date_eg_str.split('-')
    else:
        year_eg = year_in
        month_eg = month_in
        day_eg = day_in
    
    #reformat ingress time as fractional hours:
    ingress_UTC = in_hr + (in_min/60)
    egress_UTC = eg_hr + (eg_min/60)
    
    #print('\n\n\nThe longitude is set to %sdeg, (%s %sdeg %s\' %s\")' %(longitude, hemisphere, LongDeg, LongMin, LongSec))
    #TD = raw_input('\nEnter the UTC time and date as MMDDYY HHMM. (UTC = EST+5, EDT+4):\n')

    LST_in = LST_finder(int(year_in), int(month_in), int(day_in), ingress_UTC, longitude, ingress)
    LST_eg = LST_finder(int(year_eg), int(month_eg), int(day_eg), egress_UTC, longitude, egress)
    
    RAhr_obj, RAmin_obj, RAsec_obj = RA_obj.split(':')
    RA_obj = int(RAhr_obj) + (int(RAmin_obj)/60) + (float(RAsec_obj)/360)
    
    HA_in = LST_in - RA_obj
    HA_eg = LST_eg - RA_obj
    
    #HA_in_hr = int(HA_in)
    #HA_in_min = (HA_in - int(HA_in))*60
    #HA_eg_hr = int(HA_eg)
    #HA_eg_min = (HA_eg - int(HA_eg))*60
    
    #return "%s:%s"%(HA_in_hr, HA_in_min), "%s:%s"%(HA_eg_hr, HA_eg_min)
    return "%.2f" % round(HA_in, 2), "%.2f" % round(HA_eg, 2)

In [24]:
try:
    load_file
except NameError:
    load_file = None
    
try:
    load_file_planets
except NameError:
    load_file_planets = None
    
try:
    load_file_binaries
except NameError:
    load_file_binaries = None
    
if (load_file is None) and (load_file_planets is None) and (load_file_binaries is None):
    pass

elif 'None' in str(sunrise_time):
    pass

else:
    def chronological_transits(all_transits, start_date, end_date, utc_offset, longitude):
# Identify whether each transit happens in the evening or morning in order to seperate transits
# into nights later on:
        new_all_transits = []
        for row in all_transits:
            t_date, ingress = row[3].split()
            in_hr, in_min = ingress.split(':')
            if int(in_hr) <= 12:
                time_of_day = 'morning'
            elif int(in_hr) > 12:
                time_of_day = 'evening'
            new_row = list(row)
            new_row.append(time_of_day)
            new_all_transits.append(new_row)

        day_list = []
    
        start_year, start_month, start_day = start_date.split('-')
        end_year, end_month, end_day = end_date.split('-')
    
        sdate = date(int(start_year), int(start_month), int(start_day))  # start date
        edate = date(int(end_year), int(end_month), int(end_day))  # end date

        delta = edate - sdate
    
        # Make a list of consecutive dates of the run:
        for i in range(delta.days + 1):
            day = sdate + timedelta(days=i)
            day_list.append('%s' %(day))
    
        full_sorted_date_list = []
        full_sorted_NOI_list = []
        full_sorted_RA_list = []
        full_sorted_Dec_list = []
        full_sorted_ingress_list = []
        full_sorted_HA_in_list = []
        full_sorted_midtransit_list = []
        full_sorted_egress_list = []
        full_sorted_HA_eg_list = []
        full_sorted_odd_even_list = []
    
        # Run through the list of dates and check whether any transits occur on each day:
        for index, i in enumerate(day_list):
      
            noi_list = []
            RA_list = []
            Dec_list = []
            ingress_list = []
            mid_transit_list = []
            egress_list = []
            kind_list = []
            t_date_in_list = []
            in_date_time_list = []
            HA_in_list = []
            HA_eg_list = []
            j = index + 1
        # Find transits for every day except the last (we only need morning transits on the last day)
            if index != len(day_list)-1:
                next_day = day_list[j]
                for row in new_all_transits:
                    t_date_in, ingress = row[3].split()
                # Find ingresses which happen on each day in the evening and the day after in the morning:
                    if ((any('%s'%i in t_date_in for s in row)) and (row[7] == 'evening')) \
                    or ((any('%s'%(next_day) in t_date_in for s in row)) and (row[7] == 'morning')):
                     
                        RA = row[1]
                        Dec = row[2]
                        RA_list.append(RA)
                        Dec_list.append(Dec)
                        in_date_time_list.append(row[3])
                        t_date, mid_transit = row[4].split()
                        t_date, egress = row[5].split()                    
                        noi_list.append(row[0])
                        t_date_in_list.append(t_date_in)
                    
                        HA_in, HA_eg = HA_finder('%s' %t_date_in, '%s' %RA, '%s' %ingress, '%s' %egress, utc_offset, longitude)
                        HA_in_list.append(HA_in)
                        HA_eg_list.append(HA_eg)
                        ingress_list.append(ingress)
                        mid_transit_list.append(mid_transit)
                        egress_list.append(egress)
                        kind_list.append(row[6])
                    
                # Organise the ingresses on each night in order of date/ time:
                # Use the rearranged indices to rearrange the other columns
                ingress_sorted_index = np.array(sorted(range(len(in_date_time_list)), key=in_date_time_list.__getitem__))
        
                sorted_date_list = np.array(t_date_in_list)[ingress_sorted_index]
                sorted_NOI_list = np.array(noi_list)[ingress_sorted_index]
                sorted_RA_list = np.array(RA_list)[ingress_sorted_index]
                sorted_Dec_list = np.array(Dec_list)[ingress_sorted_index]
                sorted_ingress_list = np.array(ingress_list)[ingress_sorted_index]
                sorted_HA_in_list = np.array(HA_in_list)[ingress_sorted_index]
                sorted_midtransit_list = np.array(mid_transit_list)[ingress_sorted_index]
                sorted_egress_list = np.array(egress_list)[ingress_sorted_index]
                sorted_HA_eg_list = np.array(HA_eg_list)[ingress_sorted_index]
                sorted_odd_even_list = np.array(kind_list)[ingress_sorted_index]
            
            #print('  **************************************  NIGHT BEGINNING %s  **************************************  \n' %i)
            #t = PrettyTable()
            #t.add_column("ingress date", np.array(t_date_in_list)[ingress_sorted_index])
            #t.add_column ("NOI-number", np.array(noi_list)[ingress_sorted_index])
            #t.add_column ("RA", np.array(RA_list)[ingress_sorted_index])
            #t.add_column ("Dec", np.array(Dec_list)[ingress_sorted_index])
            #t.add_column("ingress (LT)", np.array(ingress_list)[ingress_sorted_index])
            #t.add_column ("HA_ingress", np.array(HA_in_list)[ingress_sorted_index])
            #t.add_column("mid_transit (LT)", np.array(mid_transit_list)[ingress_sorted_index])
         #t.add_column("egress (LT)", np.array(egress_list)[ingress_sorted_index])
            #t.add_column ("HA_egress", np.array(HA_eg_list)[ingress_sorted_index])
            #t.add_column("Odd/even", np.array(kind_list)[ingress_sorted_index])
            #print(t)
            #print('\n')
        
            else:
                pass
        
            full_sorted_date_list.append(list(sorted_date_list))
            full_sorted_NOI_list.append(list(sorted_NOI_list))
            full_sorted_RA_list.append(list(sorted_RA_list))
            full_sorted_Dec_list.append(list(sorted_Dec_list))
            full_sorted_ingress_list.append(list(sorted_ingress_list))
            full_sorted_HA_in_list.append(list(sorted_HA_in_list))
            full_sorted_midtransit_list.append(list(sorted_midtransit_list))
            full_sorted_egress_list.append(list(sorted_egress_list))
            full_sorted_HA_eg_list.append(list(sorted_HA_eg_list))
            full_sorted_odd_even_list.append(list(sorted_odd_even_list))
            
        return(full_sorted_date_list, full_sorted_NOI_list, full_sorted_RA_list, full_sorted_Dec_list, full_sorted_ingress_list, full_sorted_HA_in_list, full_sorted_egress_list, full_sorted_HA_eg_list)

    # Order transits chronologically by night beginning:

    #if ('planets' in checkbox_list) and ('binaries' in checkbox_list):
    
    #    all_transits_planets
    #    all_transits_binaries_primary
    
    #    if 'yes' in inc_secondary_transits:
    #        all_transits_binaries_secondary
    #    else:
    #        pass
    
    #else:
    #    all_transits
    #    date_list, NOI_list, RA_list, Dec_list, ingress_list, HA_in_list, egress_list, HA_eg_list = (chronological_transits(all_transits,'%s'%(start_date_val.value), '%s'%(end_date_val.value), text_utc_offset.value, longitude_widget.value))

<h4>Click 'Save' to sort transits into chronological order and save them into csv files dependant on night beginning. The output will be saved in the current directory under folders called "transits_starting_(start date)_(object_type)":</h4>

In [25]:
#date_list, NOI_list, RA_list, Dec_list, ingress_list, HA_in_list, egress_list, HA_eg_list = (chronological_transits(all_transits_list[1],'%s'%(start_date_val.value), '%s'%(end_date_val.value), text_utc_offset.value, longitude_widget.value))

#print(all_transits_list[1])

#for i in range(0,len(date_list)-1):
#    rows = zip(date_list[i], NOI_list[i], RA_list[i], Dec_list[i], ingress_list[i],  egress_list[i], HA_in_list[i], HA_eg_list[i])
#    for row in rows:
#        print(row)

In [26]:
#print(all_transits_list)
#print(inc_secondary_transits)

#for item in all_transits_list:
#    print(item)

#item,'%s'%(start_date_val.value), '%s'%(end_date_val.value), text_utc_offset.value, longitude_widget.value))

def save_transits(label, all_transits, start_date, end_date, utc_offset, longitude):
    date_list, NOI_list, RA_list, Dec_list, ingress_list, HA_in_list, egress_list, HA_eg_list = (chronological_transits(all_transits, start_date, end_date, utc_offset, longitude))
    try:
        os.mkdir('transits_starting_%s%s'%(date_list[0][0], label))
    except:
        print('Could not create directory. Perhaps one already exists called \'transits_starting_%s%s\'?'%(date_list[0][0], label))

    for i in range(0,len(date_list)-1):
        rows = zip(date_list[i], NOI_list[i], RA_list[i], Dec_list[i], ingress_list[i],  egress_list[i], HA_in_list[i], HA_eg_list[i])
        with open('transits_for_%s%s.csv'%(date_list[i][0], label), 'w') as f:
            writer = csv.writer(f)
            for row in rows:
                writer.writerow(row)
            shutil.move('transits_for_%s%s.csv'%(date_list[i][0], label),'transits_starting_%s%s'%(date_list[0][0], label))
    print('Transit lists for %s saved!'%(label[1:]))

button_save = widgets.Button(description='Save')

out_save = widgets.Output()
def on_button_clicked(_):
    #display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()-3, IPython.notebook.get_selected_index()+1)'))
    # "linking function with output"
    with out_save:
    # what happens when we press the button
        clear_output()
        if len(all_transits_list) == 1:
            if 'planets' in checkbox_list:
                label = '_planets'
            else:
                label = '_binaries'
            save_transits(label, all_transits_list[0],'%s'%(start_date_val.value), '%s'%(end_date_val.value), text_utc_offset.value, longitude_widget.value)
        else:
            for index, item in enumerate(all_transits_list):
                # For list items 1, 3, 5 etc (i.e. the data not labels)
                if index % 2 != 0:
                    save_transits('_%s'%(all_transits_list[index-1]), item,'%s'%(start_date_val.value), '%s'%(end_date_val.value), text_utc_offset.value, longitude_widget.value)
        
# linking button and function together using a button's method
button_save.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([button_save,out_save])

In [27]:
#zero_RA_sun = datetime.date(2020,3,21)
date_RA_sun = datetime.date(2021,4,17)
    
#delta = date_RA_sun - zero_RA_sun

jan_2000_RA_sun = datetime.date(2000,1,1)

delta = date_RA_sun - jan_2000_RA_sun
n = delta.days
    
L = 280.460 + (0.9856474*n)
g = 357.528 + (0.9856003*n)

L = L - ( (int(L/360)) *360)
g = g - ( (int(g/360)) *360)

lamda = L + (1.915*(np.sin(g))) + (0.02*(np.sin(2*g)))

RA = lamda + (2.45 * (np.sin(2 * lamda)))

if str(RA)[0] == '-':
    rs, RA = '-', abs(RA)
    
raH = int(RA/15)
raM = int(((RA/15)-raH)*60)

if round:
    raS = int(((((RA/15)-raH)*60)-raM)*60)
else:
    raS = ((((RA/15)-raH)*60)-raM)*60

if raH >= 12:
    raH = raH - 12
    
elif raH < 12:
    raH = raH + 12
    
midnight_RA = "%s:%s:%s" %(format(raH, '02.0f'), format(raM, '02.0f'), format(raS, '02.0f'))

#print(midnight_RA)